In [27]:
import requests
from bs4 import BeautifulSoup
import time
import re
import sqlite3

In [28]:
# 結果を保存するリスト
repositories = []

# Google組織のリポジトリ一覧ページ（元のURLを使用）
base_url = "https://github.com/orgs/google/repositories"
print(f"アクセス先: {base_url}")

# 元のコードで指定された詳細なヘッダー
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
    'Accept-Language': 'ja,en-US;q=0.9,en;q=0.8',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'none',
    'Cache-Control': 'max-age=0'
}

# セッションを使用
session = requests.Session()
session.headers.update(headers)

アクセス先: https://github.com/orgs/google/repositories


In [ ]:
# ページ番号を変えながらリポジトリを取得
page = 1
max_pages = 94  # 元の設定

print("--- 処理開始 ---")

while page <= max_pages:
    # URLを構築
    if page == 1:
        url = f"{base_url}?tab=repositories"
    else:
        url = f"{base_url}?page={page}&tab=repositories"
    
    print(f"\n[ページ {page}] {url}")
    
    try:
        # --- 1. リクエスト処理 ---
        res = session.get(url, timeout=15)
        print(f"  ステータスコード: {res.status_code}")
        
        # 503エラー処理（元のコードのロジック）
        if res.status_code == 503:
            print("  503エラー: サービス利用不可。待機時間を増やして再試行します...")
            time.sleep(5)
            res = session.get(url, timeout=15)
            print(f"  再試行後のステータスコード: {res.status_code}")
        
        # その他のエラーステータスなら例外を投げてexceptへ
        if res.status_code != 200:
            raise Exception(f"アクセスできませんでした (Status: {res.status_code})")
        
        res.encoding = 'utf-8'
        soup = BeautifulSoup(res.content, 'html.parser')
        
        # --- 2. リポジトリの検索（元のコードのロジック） ---
        repo_items = soup.find_all('li', class_='ListItem-module__listItem--k4eMk')
        
        if not repo_items:
            repo_items = soup.find_all('li', {'data-testid': lambda x: x and 'repository' in str(x).lower()})
        
        if not repo_items:
            repo_items = soup.find_all('li')
            repo_items = [item for item in repo_items if item.find('a', href=lambda x: x and '/google/' in str(x))]
        
        if not repo_items:
            print("  :warning: リポジトリが見つかりませんでした")
            # ここでbreakせずとも、finallyでページが進むので自然にループ継続または終了判断へ
            # 元のコードに合わせてbreakしたい場合はここでbreak
            break 
        
        print(f"  ✓ {len(repo_items)}個のリポジトリ要素を発見")
        
        # --- 3. データ抽出（元のコードのロジック） ---
        for item in repo_items:
            try:
                # リポジトリ名
                title_h4 = item.find('h4', class_='Title-module__heading--s7YnL')
                if title_h4:
                    repo_link = title_h4.find('a', class_='Title-module__anchor--GmXUE')
                    repo_name = repo_link.get_text().strip() if repo_link else None
                else:
                    repo_link = item.find('a', href=lambda x: x and x.startswith('/google/'))
                    repo_name = repo_link.get_text().strip() if repo_link else None
                
                if not repo_name: continue

                # 言語
                language = "不明"
                language_span = item.find('span', class_='ReposListItem-module__Text_4--mkG7R')
                if language_span:
                    language = language_span.get_text().strip()
                else:
                    language_span = item.find('span', {'itemprop': 'programmingLanguage'})
                    if language_span:
                        language = language_span.get_text().strip()
                    else:
                        lang_circle = item.find('div', class_=lambda x: x and 'LanguageCircle' in str(x))
                        if lang_circle and lang_circle.parent:
                            next_span = lang_circle.find_next_sibling('span')
                            if next_span: language = next_span.get_text().strip()

                # スター数
                star_count = "0"
                star_link = item.find('a', {'aria-label': lambda x: x and 'star' in str(x).lower()})
                if star_link:
                    star_count = star_link.get('aria-label', '').split()[0].replace(',', '')
                else:
                    star_link = item.find('a', href=lambda x: x and 'stargazers' in str(x))
                    if star_link:
                        star_count = star_link.get_text().strip().replace(',', '')
                    else:
                        star_svg = item.find('svg', class_='octicon-star')
                        if star_svg and star_svg.parent:
                            star_text = star_svg.parent.get_text().strip()
                            match = re.search(r'[\d,]+', star_text)
                            if match: star_count = match.group().replace(',', '')

                # 保存
                repositories.append({
                    'リポジトリ名': repo_name,
                    '主要な言語': language,
                    'スター数': star_count
                })
                print(f"    ✓ {repo_name} | {language} | {star_count}")

            except Exception as e:
                # 個別の抽出エラーは無視して次へ
                continue
                
    except Exception as e:
        print(f"  → エラーが発生しました: {e}")
        # import traceback
        # traceback.print_exc()
        break # 致命的なエラーならループを抜ける

    finally:
        # 【重要】ここが追加機能です
        # 成功しても失敗しても必ず実行される処理
        
        # 次のページへ進む
        page += 1
        
        # 待機
        print("  (待機中...)")
        time.sleep(2)

--- 処理開始 ---

[ページ 1] https://github.com/orgs/google/repositories?tab=repositories
  ステータスコード: 200
  ✓ 30個のリポジトリ要素を発見
    ✓ skia | C++ | 10k
    ✓ device-infra | Java | 58
    ✓ budoux-extension | TypeScript | 16
    ✓ nearby | C++ | 887
    ✓ qwix | Python | 64
    ✓ aarch64-rt | Rust | 17
    ✓ percore | Rust | 8
    ✓ wuffs | C | 4.7k
    ✓ magika | Python | 9.8k
    ✓ jimfs | Java | 2.5k
    ✓ zerocopy | Rust | 2.1k
    ✓ skia-buildbot | Go | 158
    ✓ XNNPACK | C | 2.2k
    ✓ gtm-session-fetcher | Objective-C | 265
    ✓ google-toolbox-for-mac | Objective-C | 1.2k
    ✓ ground-android | Kotlin | 272
    ✓ command-fds | Rust | 42
    ✓ yapf | Python | 14k
    ✓ angle | C++ | 3.8k
    ✓ tsl | C++ | 101
    ✓ osv-scalibr | Go | 537
    ✓ synopsys-dw-uart | Rust | 1
    ✓ cameratrapai | Python | 386
    ✓ double-conversion | C++ | 1.2k
    ✓ capslock | Go | 1.1k
    ✓ dive | C++ | 17
    ✓ tunix | Python | 1.9k
    ✓ adk-samples | Python | 6.4k
    ✓ sedpack | Python | 28
    ✓ go-con

In [ ]:
# 結果を表示
print("\n" + "="*60)
print(f"取得したリポジトリ数: {len(repositories)}")
print("="*60)

for i, repo in enumerate(repositories[:20], 1):  # 最初の20件を表示
    print(f"{i}. {repo['リポジトリ名']}")
    print(f"   言語: {repo['主要な言語']}")
    print(f"   スター: {repo['スター数']}")
    print()